In [1]:
from aframe import AFrame
import pandas as pd

In [2]:
af = AFrame(dataverse='test', dataset='TweetItems')

In [3]:
af

AsterixDB DataFrame with the following known columns: 
	[{'name': 'id', 'type': 'int64', 'nullable': False}, {'name': 'text', 'type': 'string', 'nullable': False}]

In [4]:
len(af)

5981

In [5]:
af.columns

"[{'name': 'id', 'type': 'int64', 'nullable': False}, {'name': 'text', 'type': 'string', 'nullable': False}]"

In [6]:
df = af.toPandas()

In [7]:
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
0,NaN,None,2018-02-08 05:18:41,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 961400756740923392, 'id_str'...",NaN,0,False,low,...,0,0,False,{'created_at': 'Thu Feb 08 00:46:19 +0000 2018...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @MicahGrimes: 2 iconic photos. 1 year. Only...,2018-02-08 05:18:41.604,False,"{'id': 320000200, 'id_str': '320000200', 'name...",NaN
1,NaN,None,2018-02-08 05:18:41,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,0,0,False,{'created_at': 'Wed Feb 07 22:54:04 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @EdKrassen: Nancy Pelosi has more Enerygy a...,2018-02-08 05:18:41.705,False,"{'id': 50667682, 'id_str': '50667682', 'name':...",NaN
2,NaN,None,2018-02-08 05:18:41,"[0, 140]","{'hashtags': [{'text': 'POTUS', 'indices': [66...",NaN,{'full_text': 'CNN - INTERNATIONAL . THE LEAST...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",CNN - INTERNATIONAL . THE LEAST TRUSTED NAME...,2018-02-08 05:18:41.713,True,"{'id': 882677034429710336, 'id_str': '88267703...",NaN
3,NaN,None,2018-02-08 05:18:41,"[17, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': '@realDonaldTrump The Russian As...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump The Russian Asset Donald Trum...,2018-02-08 05:18:41.773,True,"{'id': 958747420904497153, 'id_str': '95874742...",NaN
4,NaN,None,2018-02-08 05:18:41,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,0,0,False,{'created_at': 'Wed Feb 07 15:48:03 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @nowthisnews: This Trump supporter used the...,2018-02-08 05:18:41.802,False,"{'id': 749752742, 'id_str': '749752742', 'name...",NaN


In [8]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'reply_count',
       'retweet_count', 'retweeted', 'retweeted_status', 'source', 'text',
       'timestamp_ms', 'truncated', 'user', 'withheld_in_countries'],
      dtype='object')

In [9]:
af['truncated'].head(2)

,0
0,False
1,False


In [10]:
df['truncated'].head(2)

0    False
1    False
Name: truncated, dtype: bool

In [11]:
af[af['truncated'] == True].query

'select value t from test.TweetItems t where t.truncated = True;'

In [12]:
df[df['truncated'] == True].head(2)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
2,NaN,None,2018-02-08 05:18:41,"[0, 140]","{'hashtags': [{'text': 'POTUS', 'indices': [66...",NaN,{'full_text': 'CNN - INTERNATIONAL . THE LEAST...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",CNN - INTERNATIONAL . THE LEAST TRUSTED NAME...,2018-02-08 05:18:41.713,True,"{'id': 882677034429710336, 'id_str': '88267703...",NaN
3,NaN,None,2018-02-08 05:18:41,"[17, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': '@realDonaldTrump The Russian As...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump The Russian Asset Donald Trum...,2018-02-08 05:18:41.773,True,"{'id': 958747420904497153, 'id_str': '95874742...",NaN


In [13]:
a = af['truncated']
print(a.query)

select value t.truncated from test.TweetItems t;


In [14]:
a = af['truncated'] == True
print(a.query)

select value t.truncated = True from test.TweetItems t;


In [15]:
a = af[af['truncated'] == True]
print(a.query)

select value t from test.TweetItems t where t.truncated = True;


In [16]:
a = (af['truncated'] == True) & (af['favorited'] == False)
print(a.query)

select value t.truncated = True and t.favorited = False from test.TweetItems t;


In [17]:
a.head()

,0
0,False
1,False
2,True
3,True
4,False


In [18]:
b = (df['truncated'] == True) & (df['favorited'] == False)
b.head()

0    False
1    False
2     True
3     True
4    False
dtype: bool

In [19]:
a = af[(af['truncated'] == True) & (af['favorited'] == False)]
print(a.query)

select value t from test.TweetItems t where t.truncated = True and t.favorited = False;


In [20]:
a.head(2)

,contributors,coordinates,created_at,display_text_range,entities,extended_tweet,favorite_count,favorited,filter_level,geo,...,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,NaN,2018-02-08 05:18:41,"[0, 140]","{'hashtags': [{'text': 'POTUS', 'indices': [66...",{'full_text': 'CNN - INTERNATIONAL . THE LEAST...,0,False,low,NaN,...,9.561978e+17,9.561978e+17,0,0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",CNN - INTERNATIONAL . THE LEAST TRUSTED NAME...,2018-02-08 05:18:41.713,True,"{'id': 882677034429710336, 'id_str': '88267703..."
1,NaN,NaN,2018-02-08 05:18:41,"[17, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",{'full_text': '@realDonaldTrump The Russian As...,0,False,low,NaN,...,NaN,NaN,0,0,False,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump The Russian Asset Donald Trum...,2018-02-08 05:18:41.773,True,"{'id': 958747420904497153, 'id_str': '95874742..."


In [21]:
b = df[(df['truncated'] == True) & (df['favorited'] == False)]
b.head(2)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries
2,NaN,None,2018-02-08 05:18:41,"[0, 140]","{'hashtags': [{'text': 'POTUS', 'indices': [66...",NaN,{'full_text': 'CNN - INTERNATIONAL . THE LEAST...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",CNN - INTERNATIONAL . THE LEAST TRUSTED NAME...,2018-02-08 05:18:41.713,True,"{'id': 882677034429710336, 'id_str': '88267703...",NaN
3,NaN,None,2018-02-08 05:18:41,"[17, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': '@realDonaldTrump The Russian As...,0,False,low,...,0,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump The Russian Asset Donald Trum...,2018-02-08 05:18:41.773,True,"{'id': 958747420904497153, 'id_str': '95874742...",NaN


In [22]:
df_a = a.collect()
print('AFrame cnt: %d' % len(df_a))
print('DataFrame cnt: %d' % len(b))

AFrame cnt: 564
DataFrame cnt: 564
